# Exercise 2 (Due: January 19, 2018 before midnight)

The purpose of this exercise is to help you get familiar with the Python tweepy library for downloading data from Twitter. Follow the instructions below to complete the exercise. Use the template notebook below to write and execute your python program. Rename the template file as exercise2.ipynb and submit it via D2L. You should also convert the notebook to PDF or HTML and submit the converted file as well.

**Step 1.** Create a Twitter account and create a new Twitter application using the account. Make sure you write down your consumer and access tokens and secret keys.

**Step 2.** Following the Python tweepy example given in the lecture, use the Twitter Search (REST) API to download 30 most recent tweets from the Twitter feed of “CDCgov”..  This can be done by changing the keyword string given in the sample code to “from:CDCgov”. For more information about the query term to use, read the documentation available at https://dev.twitter.com/rest/public/search. Make sure you install tweepy on your machine before running the code. Save the text messages of the returned tweets into an ASCII text file named cdc.txt. **Make sure you save only the text part of the tweet message NOT the entire JSON message** (which includes username, coordinates, etc). See the example given in the lecture notes on how to access the text part of the tweets.  The following is an example about how to save a string into a file:

with open(“filename”,”w”) as f:


    text = "this is a string" 
	f.write(text)
    f.write("\n")
    
**Solution:**

In [43]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
import config

# Replace the X's with your consumer tokens and access tokens.
# To create the application, go to https://dev.twitter.com
C_KEY = config.C_KEY
C_SECRET = config.C_SECRET
A_TOKEN_KEY = config.A_TOKEN_KEY
A_TOKEN_SECRET = config.A_TOKEN_SECRET

auth = tweepy.OAuthHandler(C_KEY, C_SECRET)
auth.set_access_token(A_TOKEN_KEY, A_TOKEN_SECRET)
api = tweepy.API(auth)

keyword = 'from:CDCgov'
posts = api.search(q=keyword, count=30)

with open('cdc.txt', 'w') as f:
    for tweet in posts:
        f.write(str(tweet.text.encode("utf-8")))
        f.write("\n")


---
**Step 3.** Use the Twitter Streaming API to download tweets containing the keyword “health”. Set the time limit to 30 seconds and store the results into a file named health.json. 

**Solution:**

In [44]:
import time
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import config

C_KEY = config.C_KEY
C_SECRET = config.C_SECRET
A_TOKEN_KEY = config.A_TOKEN_KEY
A_TOKEN_SECRET = config.A_TOKEN_SECRET

# Create a StreamListener class 
class MyListener(StreamListener):

    def __init__(self, time_limit=30):
        self.start_time = time.time()
        self.limit = time_limit
        self.outFile = open('health.json', 'a')
        super(MyListener, self).__init__()
        
    def on_data(self, data):
        if(time.time() - self.start_time) < self.limit:
            self.outFile.write(data.strip())
            self.outFile.write('\n')
            return True
        else:
            self.outFile.close()
            return False
        
    def on_error(self, status):
        print(status)
        
auth = OAuthHandler(C_KEY,C_SECRET)
auth.set_access_token(A_TOKEN_KEY,A_TOKEN_SECRET)
myStream = Stream(auth, MyListener(time_limit=10 ))
myStream.filter(track=['health'])

In [45]:
# Checking to make sure that your json file is stored correctly.
# If the json.loads command fails, modify your on_data() method to the following
# to deal with the newline character at the end of each tweet message:
#    self.outFile.write(data.strip())   
#    self.outFile.write('\n') 

import json

with open('health.json') as f:
    tweets = [json.loads(line) for line in f]
    for twt in tweets:
        print(twt['id'], 'posted', twt['text'], '\n')

(1086025615407529984, 'posted', u'RT @DellMedSchool: The 1,000 butterflies formed out of aluminum cans in Paul Villinski\u2019s sculpture, \u201cPassage,\u201d are a beautiful symbol of co\u2026', '\n')
(1086025616493813760, 'posted', u'RT @sparkIy: my mental health https://t.co/4S1EfEXRDD', '\n')
(1086025617127092224, 'posted', u'RT @hyojetion: 190117 \uc0b0\ub4e4\uc758 \ubcc4\ubc24 #\uc628\uc564\uc624\ud504 #ONF (1)\n\nONF: Lights On! Hello, we are ONF!\nHYOJIN: CNU senior! Please always take care of your health.\u2026', '\n')
(1086025617831800835, 'posted', u'RT @Politicallady9: #WheresMitch https://t.co/S7wTvICZob', '\n')
(1086025617949319170, 'posted', u'RT @voidchambie: my mom said that i could visit my grandma if i get 500 subs on yt by saturday morning. i haven\u2019t seen my grandma in two ye\u2026', '\n')
(1086025618419077120, 'posted', u'RT @NHSMillion: Still think the privatisation of health services is a myth?\n\nThen why are sexual health services in Cheshire West now pro

**Step 4:** Install MySQL server on your machine. Go to https://dev.mysql.com/downloads/ and download the installation files. After installing, create a user named 'cse482' with a password named 'cse482'. Create also a database named 'cse482'. To do this, login to the mysql client as administrator and then performed the following steps:

mysql> CREATE USER 'cse482' IDENTIFIED BY 'cse482';

mysql> CREATE DATABASE cse482;

mysql> GRANT ALL PRIVILEGES ON cse482.* TO 'cse482'@'%';

Note that the preceding commands are only available if you're the administrator of the database.

**Step 5:** Install the mysql-connector-python library. Create a mysql table named 'Health', which has the following 3 columns:

    TweetID BIGINT PRIMARY KEY,

    UserID BIGINT,
    
    Message VARCHAR(200)


In [46]:
uname = "cse482"
pwd = "cse482"
hname = "localhost"
dbname = "cse482"

import mysql.connector
from mysql.connector import errorcode

try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=dbname)
    cursor = cnx.cursor()
    query = "DROP TABLE IF EXISTS Health"
    cursor.execute(query)

    query = """ CREATE TABLE IF NOT EXISTS Health
                (TweetID BIGINT PRIMARY KEY,
                UserID BIGINT,
                Message VARCHAR(200)) 
            """
    cursor.execute(query)
    cursor.close()
    cnx.close()
    
except mysql.connector.Error as e:
    print(e.msg)       # error message

**Step 6:** Open the health.json file you've created. For each tweet, insert the tweetID, userID, and tweet text message into the Health table in MySQL.

In [47]:
try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=dbname)
    cursor = cnx.cursor()
    
    with open('health.json') as f:
        tweets = [json.loads(line) for line in f]
        for twt in tweets:
            query = """ 
                    INSERT INTO Health VALUES
                    ('{}','{}','{}')
                    """.format(twt['id'],twt['user']['id'],str(twt['text'].encode("utf-8")).replace("'","\''"))
            cursor.execute(query)
        
    cnx.commit()
    cursor.close()
    cnx.close()
    
except mysql.connector.Error as e:
    print(e.msg)       # error message

In [48]:
try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=dbname)
    cursor = cnx.cursor()

    query = """ 
                SELECT Message from cse482.Health
            """
    cursor.execute(query)
    
    for row in cursor:
        print(row)
        
    cursor.close()
    cnx.close()
    
except mysql.connector.Error as e:
    print(e.msg)       # error message

(u'RT @DellMedSchool: The 1,000 butterflies formed out of aluminum cans in Paul Villinski\u2019s sculpture, \u201cPassage,\u201d are a beautiful symbol of co\u2026',)
(u'RT @sparkIy: my mental health https://t.co/4S1EfEXRDD',)
(u'RT @hyojetion: 190117 \uc0b0\ub4e4\uc758 \ubcc4\ubc24 #\uc628\uc564\uc624\ud504 #ONF (1)\n\nONF: Lights On! Hello, we are ONF!\nHYOJIN: CNU senior! Please always take care of your health.\u2026',)
(u'RT @Politicallady9: #WheresMitch https://t.co/S7wTvICZob',)
(u'RT @voidchambie: my mom said that i could visit my grandma if i get 500 subs on yt by saturday morning. i haven\u2019t seen my grandma in two ye\u2026',)
(u'RT @NHSMillion: Still think the privatisation of health services is a myth?\n\nThen why are sexual health services in Cheshire West now provi\u2026',)
(u'RT @GoTeamMH: Thank you @normanlamb for highlighting the vital importance of proactive preventative whole organisation approaches to mental\u2026',)
(u'RT @SenGillibrand: .@senatemajldr has schedu